# Electrode to ROI Projection Function
This notebook defines a function `compute_projection_weights` that takes electrode MNI coordinates, mirrors right hemisphere to the left, and returns normalized projection weights to 8 target ROIs from the AAL atlas.

In [1]:
!pip install nibabel nilearn --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.6/10.6 MB 41.2 MB/s eta 0:00:00


In [23]:
import numpy as np
import nibabel as nib
from scipy.ndimage import gaussian_filter
# from nilearn.datasets import fetch_atlas_aal
from nilearn.image import load_img
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')

path = "/content/drive/MyDrive/aal/aal2.nii.gz"

# Load AAL atlas and metadata
# aal_atlas = fetch_atlas_aal()
aal_img = aal_img = nib.load(path)
aal_data = aal_img.get_fdata()
affine = aal_img.affine
with open("/content/drive/MyDrive/aal/aal2.nii.txt") as f:
    aal_labels = [line.strip().split()[1] for line in f]
aal_dict = {i+1: label for i, label in enumerate(aal_labels)}
# aal_labels = aal_atlas['labels']
# aal_dict = {i+1: label for i, label in enumerate(aal_labels)}
atlas_shape = aal_data.shape
print(aal_dict)
print(np.unique(aal_data))
# Define 8 target ROIs
target_labels = [
    'Frontal_Mid_2_L',
    'Precentral_L',
    'Postcentral_L',
    'Parietal_Inf_L',
    'SupraMarginal_L',
    'Temporal_Sup_L',
    'Temporal_Mid_L',
    'Temporal_Inf_L'
]
# target_labels = [v for v in aal_dict.values() if any(k in v for k in target_rois)]
fwhm_mm = 20
sigma_vox = fwhm_mm / (2.355 * np.mean(np.diag(affine[:3, :3])))

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
{1: 'Precentral_L', 2: 'Precentral_R', 3: 'Frontal_Sup_2_L', 4: 'Frontal_Sup_2_R', 5: 'Frontal_Mid_2_L', 6: 'Frontal_Mid_2_R', 7: 'Frontal_Inf_Oper_L', 8: 'Frontal_Inf_Oper_R', 9: 'Frontal_Inf_Tri_L', 10: 'Frontal_Inf_Tri_R', 11: 'Frontal_Inf_Orb_2_L', 12: 'Frontal_Inf_Orb_2_R', 13: 'Rolandic_Oper_L', 14: 'Rolandic_Oper_R', 15: 'Supp_Motor_Area_L', 16: 'Supp_Motor_Area_R', 17: 'Olfactory_L', 18: 'Olfactory_R', 19: 'Frontal_Sup_Medial_L', 20: 'Frontal_Sup_Medial_R', 21: 'Frontal_Med_Orb_L', 22: 'Frontal_Med_Orb_R', 23: 'Rectus_L', 24: 'Rectus_R', 25: 'OFCmed_L', 26: 'OFCmed_R', 27: 'OFCant_L', 28: 'OFCant_R', 29: 'OFCpost_L', 30: 'OFCpost_R', 31: 'OFClat_L', 32: 'OFClat_R', 33: 'Insula_L', 34: 'Insula_R', 35: 'Cingulate_Ant_L', 36: 'Cingulate_Ant_R', 37: 'Cingulate_Mid_L', 38: 'Cingulate_Mid_R', 39: 'Cingulate_Post_L', 40: 'Cingulate_Post_R', 41: 'Hippocampus_

In [24]:
def compute_projection_weights(electrode_xyz):
    weights_all = []
    for coord in electrode_xyz:
        if coord[0] > 0:
            coord[0] = -coord[0]  # Mirror to left hemisphere
        voxel_coord = np.linalg.inv(affine).dot(np.append(coord, 1))[:3] # Changing coord to aal voxel using affine
        voxel_coord = np.round(voxel_coord).astype(int) # to round
        gauss_volume = np.zeros(atlas_shape) # initialization
        try:
            gauss_volume[tuple(voxel_coord)] = 1 # Gaussian center
            gauss_volume = gaussian_filter(gauss_volume, sigma=sigma_vox) # sigma: radius - mm changed to voxel
        except IndexError:
            gauss_volume[:] = 0
        weights = {}
        for i in np.unique(aal_data):   # loop for all ROIs in AAL atlas
            if i == 0: continue
            label = aal_dict[int(i)]   # find ROI name
            if label not in target_labels:
                continue
            roi_weight = gauss_volume[aal_data == i].sum()
            weights[label] = roi_weight
        total = sum(weights.values())
        norm_weights = [weights.get(roi, 0) / total if total > 0 else 0 for roi in target_labels]
        weights_all.append(norm_weights)
    return pd.DataFrame(weights_all, columns=target_labels)

In [25]:
# Example coordinates
example_coords = np.array([
    [30, -20, 60],
    [-40, -30, 55],
    [45, -15, 50]
])
df_proj = compute_projection_weights(example_coords)
df_proj

,Frontal_Mid_2_L,Precentral_L,Postcentral_L,Parietal_Inf_L,SupraMarginal_L,Temporal_Sup_L,Temporal_Mid_L,Temporal_Inf_L
0,0.098730,0.335644,0.334200,0.144550,0.044550,0.024131,0.016815,0.001381
1,0.050255,0.246682,0.337888,0.210620,0.073486,0.040573,0.037354,0.003143
2,0.113949,0.296597,0.300703,0.127235,0.070885,0.051011,0.035689,0.003932
